In [1]:
import hashlib
from PIL import Image
import io, os
import requests
import time
from selenium import webdriver

In [7]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception as e:
                print(e)
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            #return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

def search_and_download(search_term:str,driver_path=DRIVER_PATH,target_path=t_path,number_images=no_img):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    for elem in res:
        persist_image(target_folder,elem)

In [17]:
t_path = './images1'

DRIVER_PATH = '/Users/rameshveer/Desktop/chromedriver'

no_img = 75

search_and_download('heavy workers wearing saftey shoes', DRIVER_PATH, t_path, no_img)

Found: 95 search results. Extracting links from 0:95
Found: 75 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTruCxdRCfJfojR50n0iOfr1TwD036bojXhOg&usqp=CAU - as ./images1/chemical_workers_wearing_masks/a36eb65a8c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSU7hqW3-WtQjKeetFS_9rS9b43jyYTw3GYpg&usqp=CAU - as ./images1/chemical_workers_wearing_masks/cca94c3000.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR0ibRhLEZPA0GLCRYMAjRW2n-uNVVLaQ-ZTA&usqp=CAU - as ./images1/chemical_workers_wearing_masks/f43f65eca2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT_suxSlEcigjOAMc0p9mD80imBdVlZeGQgCg&usqp=CAU - as ./images1/chemical_workers_wearing_masks/046acbc8e1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcStHdRiYh8NTnrk5rIUylLvLpO_8Zrebbujwg&usqp=CAU - as ./images1/chemical_workers_wearing_masks/2275474b36.jpg
SUCCESS - saved https:/

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ_B4_2D95fGDRk1DgMgo0uvWBYyI3yv633kA&usqp=CAU - as ./images1/chemical_workers_wearing_masks/e9673ec729.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSLaYfxwj2uRD34e4p5kzwZrykZbOfCDqAjxA&usqp=CAU - as ./images1/chemical_workers_wearing_masks/adb4384d90.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcThx3hbqiuiJ3dSMTArvAsetkZS44Mzwd0BIg&usqp=CAU - as ./images1/chemical_workers_wearing_masks/3fe7e774df.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRAPhUaaQzP4wAag2tK3GER9dkmXFbA_F_mVQ&usqp=CAU - as ./images1/chemical_workers_wearing_masks/fce40919b8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRHvHnNfc3ptKWV6EhMV5_EJLV4BGagvTeX2A&usqp=CAU - as ./images1/chemical_workers_wearing_masks/de32a0dafc.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRTLrEfjEeH2gUDvc7zDLSrkUOiph6cO22